In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import spacy
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reading the full sample into the DataFrame full_sample. I decided to cut 'created_at' as I did not see an immediate use for it.

In [ ]:
full_sample = pd.read_csv("/kaggle/input/tweets-data/twcs.csv")
full_sample.drop(columns=['created_at'], inplace=True)
EMBEDDING_FILE = '/kaggle/input/glove840b300dtxt/glove.840B.300d.txt'

At the beginning of the project for the sake of load times I used the sample csv to test my functions and explore the dataset. I begun with loading the data into the kaggle document and some inital EDA.

In [ ]:
sample = pd.read_csv("/kaggle/input/sample/sample.csv")
sample.info()
sample.describe()
sample.head(10)
sample['author_id'].unique()
sample.drop(columns=['created_at'], inplace=True)
sample.head()

This is my full preprocessing condensed into one segment for easy of use. Full steps are below. This will take awhile, feel free to reduce the question_f.head(5000) to something smaller. It should all work fine. I hope. For reference my computer takes just over 3 minutes.

In [ ]:
full_sample = pd.read_csv("/kaggle/input/tweets-data/twcs.csv")
full_sample.drop(columns=['created_at'], inplace=True)

questions = full_sample[full_sample["inbound"] == True]
#questions.head()
answers = full_sample[full_sample["inbound"] == False]
#answers.head()

question_f = pd.DataFrame(data=questions, columns = ['tweet_id', 'author_id', 'inbound', 'text', 'response_tweet_id', 'in_response_to_tweet_id'])
answer_f = pd.DataFrame(data=answers)
#question_f

question_f = question_f[question_f['response_tweet_id'].notna()]
#sample['text']
question_f = question_f.head(5000)
#question_f.info()

responses = question_f['response_tweet_id']
#print(responses)
new_responses = []
checker = ","
for response in responses: 
    if checker in response:
        response_parts = response.split(",")
        one_response = response_parts[0]
        new_responses.append(one_response)
    else:
        new_responses.append(response)   
    
#print(len(new_responses))

#print(new_responses)
print("1/5")
tweets = []
#error = 'Series([], Name: text, dtype: object)'
for r in new_responses:
    
    #print(type(r))
    number = int(r)
    tweet = answer_f[answer_f['tweet_id'] == number]['text'].values
    #print((tweet))
    if len(tweet) == 0:
        tweet = None
    tweets.append(tweet)
    
    
question_f['response_tweet'] = tweets
#question_f = question_f.drop(question_f[question_f.response_tweet == None].index)
#question_f.replace({'NaN': None})
question_f = question_f[question_f['response_tweet'].notna()]
#question_f

print("2/5")
!pip install spacy_cld

import spacy
from spacy_cld import LanguageDetector

nlp = spacy.load('en')
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)

s = question_f['text'].values
lang_list = []

for string in s:
    doc = nlp(string)
    
    if(doc._.languages):
        lang_list.append(doc._.languages[0])
    # if it is empty, we append the list by unknown
    else:
        lang_list.append('unknown')

question_f['languages_spacy'] = lang_list

question_f = question_f.drop(question_f[question_f.languages_spacy != 'en'].index)

print("3/5")

import emoji
import spacy
import stop_words
s = question_f['text'].values
#print(s)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

list_of_list = []

nlp = spacy.load("en_core_web_sm")
for string in s:
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:8] == "https://":
            my_list += 'hyperlink' + ' '
        elif token.text[0:1] == "@":
            my_list += 'company' + ' '
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
                my_list += thisdict["lem"] + ' '
       
    list_of_list.append(my_list)

print("4/5")
#print(list_of_list)
question_f['processing_inti'] = list_of_list
#question_f.head()

question_f['response_tweet'] = question_f['response_tweet'].astype(str)
s = question_f['response_tweet']
#print(type(s))

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

list_of_lists = []

nlp = spacy.load("en_core_web_sm")
for string in s:
    #print(string)
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:8] == "https://":
            my_list += 'hyperlink' + ' '
        elif token.text[0:1] == "@":
            my_list += 'user' + ' '
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
                my_list += thisdict["lem"] + ' '
       
    list_of_lists.append(my_list)
    
#print(list_of_list)
question_f['processing_resp'] = list_of_lists
print('5/5')

#cleaning up the dataframe a bit
question_f.drop(columns=['inbound'], inplace=True)
question_f.drop(columns=['in_response_to_tweet_id'], inplace=True)
question_f.drop(columns=['languages_spacy'], inplace=True)
question_f.drop(columns=['response_tweet_id'], inplace=True)
question_f.drop(columns=['author_id'], inplace=True)
question_f.head()

Word cloud to explore common words in the processed question-answer pairs.

In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

#mpl.rcParams['figure.figsize']=(8.0,6.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1 


stopwords = set(STOPWORDS)
data = question_f['text'].values

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=50,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(question_f['processing_resp'].values))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=900)

In [ ]:
question_f.head(5)

At this point I was a bit confused about what form the data I needed was supposed to be in. I found an FAQ dataset for an E-learning system used for a japanese university. This chatbot took in three datasets: questions, answers and catagories. I'm unsure on how to I'll do the catagories but the first two should be easy enough. I noticed that "inbound" could be used to create question - answer pairs. There were some NaN values in 'response_tweet_id' so I just removed them.

In [ ]:
questions = full_sample[full_sample["inbound"] == True]
#questions.head()
answers = full_sample[full_sample["inbound"] == False]
answers.head()

question_f = pd.DataFrame(data=questions, columns = ['tweet_id', 'author_id', 'inbound', 'text', 'response_tweet_id', 'in_response_to_tweet_id'])
answer_f = pd.DataFrame(data=answers)
question_f

question_f = question_f[question_f['response_tweet_id'].notna()]
#sample['text']
question_f = question_f.head(5000)
question_f.info()

I'm now going to pair the tweet with the reponse_tweet_id to create a question - answer pair for my chatbot. Some of the tweets have multiple responses, annyoing, I can either create new rows to form new question-response pairs or remove one of the reponses. I know which one is easier but I'll try creating new pairs first. 
After more thought, I realised these duplicates would mess with the vocab building, so I'm opting for the dirty method and will remove the excess responses. Aka, create a new list of response_tweet_id's using the first id in each row.
The objects were strings so manipulation should be easy enough.

In [ ]:
responses = question_f['response_tweet_id']
#print(responses)
new_responses = []
checker = ","
for response in responses: 
    if checker in response:
        response_parts = response.split(",")
        one_response = response_parts[0]
        new_responses.append(one_response)
    else:
        new_responses.append(response)   
    
#print(len(new_responses))

#print(new_responses)

Now I just have to find the corrosponding tweets in my answers dataframe. Some of the response tweets weren't in the CSV file so they have to be removed. 

In [ ]:
answer_f
one = answer_f[answer_f['tweet_id'] == 119240]['text']
print(one)

In [ ]:
#answer_f.head()
tweets = []
for r in new_responses:
    
    #print(type(r))
    number = int(r)
    tweet = answer_f[answer_f['tweet_id'] == number]['text'].values
    #print((tweet))
    if len(tweet) == 0:
        tweet = None
    tweets.append(tweet)
    
#print(len(tweets))
#print(tweets[1])
#s = tweets[1]
#print(s)

question_f['response_tweet'] = tweets
#question_f = question_f.drop(question_f[question_f.response_tweet == None].index)
#question_f.replace({'NaN': None})
question_f = question_f[question_f['response_tweet'].notna()]
#question_f

In [ ]:
question_f.info()

I noticed later on that there were some foreign languages in the twitter database. Of course this won't be much help to my chatbot so I can either translate them or delete them. 

In [ ]:
!pip install spacy_cld

This is a simple loop that creates a list with the primary language of each 'doc' aka tweet that I will use to remove all but English.

In [ ]:
import numpy as np 
import pandas as pd
import re
import spacy
from spacy_cld import LanguageDetector

nlp = spacy.load('en')
#language_detector = LanguageDetector()
nlp.add_pipe(language_detector)

s = question_f['text'].values
lang_list = []

for string in s:
    doc = nlp(string)
    
    if(doc._.languages):
        lang_list.append(doc._.languages[0])
    # if it is empty, we append the list by unknown
    else:
        lang_list.append('unknown')

question_f['languages_spacy'] = lang_list


After finding a number of small occurances outside of the main few it seems easier to remove all but the 'en', English, tag. I used the built in drop function for this.

In [ ]:
question_f['languages_spacy'].value_counts()
question_f = question_f.drop(question_f[question_f.languages_spacy != 'en'].index)

Now onto tagging and categorizing the words. I started by using the method you showed us using NLTK, confused myself trying to manipulate strings, and ended up building my own method using spacy. This method removes stopwords, punctuation and symbols and provides lemmatization. I came back to change this process from removing punctuation and symbols to keeping nouns, verbs, proper nouns and numbers. I'm unsure whether this makes the remove stopwords redundant but I can't harm to keep it in. (except in computing time; oops)

In [ ]:
import emoji
import spacy
import stop_words
s = question_f['text']
#print(s)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

list_of_list = []

nlp = spacy.load("en_core_web_sm")
for string in s:
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:8] == "https://":
            my_list += 'hyperlink' + ' '
        #print(type(token))
        elif token.text[0:1] == "@":
            my_list += token.text[1:] + ' '
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM')):
                my_list += thisdict["lem"] + ' '
       
    list_of_list.append(my_list)
    
#print(list_of_list)
question_f['processing_inti'] = list_of_list
question_f.head()

This is the same method but for the responses. I could combine them to create a function but this is low on my TODO list.

In [ ]:
import emoji
import spacy
import stop_words
question_f['response_tweet'] = question_f['response_tweet'].astype(str)
s = question_f['response_tweet']
#print(type(s))

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

list_of_lists = []

nlp = spacy.load("en_core_web_sm")
for string in s:
    #print(string)
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:8] == "https://":
            my_list += 'hyperlink' + ' '
        elif token.text[0:1] == "@":
            my_list += 'user' + ' '        
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM')):
                my_list += thisdict["lem"] + ' '
       
    list_of_lists.append(my_list)
    
#print(list_of_list)
question_f['processing_resp'] = list_of_lists
question_f.head()

With my question - answer pairs sorted I decided to create a very simple chatbot that 
1. Processes input data
2. Loops through the existing data 
3. Compares the similarity of the two vector averages
4. Finds an existing question with a similarity > 0.8
5. Finds the corrosponding answer and prints it

In [ ]:
import spacy
#from spacy import en_core_web_sm
flag = True
questions_t = list(question_f['processing_inti'].values)
answers_t = list(question_f['processing_resp'].values)

def process_string(user_input):
    nlp = spacy.load("en_core_web_sm")
    my_list = ""
    doc = nlp(user_input)   
    for token in doc:        
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & (thisdict["pos"] != 'PUNCT') & (thisdict["pos"] != 'SYM'):
            my_list += thisdict["lem"] + ' '
       
    return my_list

print('BOT: My name is ChatChat.v1, I will answer your questions about as best as i can. If you want to exit, type Bye')

while flag:
    user_response = input( 'Enter question : ' )
    user_response = user_response.lower()
    print("User: ", user_response)
    answered = 0
    if user_response is not 'bye':
        processed_input = process_string(user_response)
        
        doc1 = nlp(processed_input)
        for quest in questions_t:
            doc2 = nlp(quest)
            simil = doc1.similarity(doc2)
            #add randomiser to retrieve random responses instead of the first one
            if simil > 0.80:
                index = questions_t.index(quest)
                print("ChatChat: ", answers_t[index])
                answered = 1
                break;
        if answered == 0:
            print("Sorry I can't understand you. Please try again.")        
    else:
        print('ChatChat: bye!')
        flag = False

I will now create a more advanced chatbot, building from what we have.

First I'm going to quickly edit my list of inputs to a max length = 20 tokens, min length = 3 tokens.

In [ ]:
sentences = list(question_f['processing_inti'].values)
answers_pre =  list(question_f['processing_resp'].values)
max_length = 21
min_length = 2
questions = list()
answers = list()
for sent in sentences:
    length = len(sent.split())
    #print(len(tokens))
    if length > min_length & length < max_length:
        index = sentences.index(sent)
        questions.append(sentences[index])
        answers.append(answers_pre[index])
        
#print(len(questions))
#print(len(answers))
questions_tokened = list()
for q in questions:    
    tokens = list(q.split())
    questions_tokened.append(tokens)
    
print(len(questions_tokened))
print(questions_tokened[1])

There are two methods for my next step, i'll either use doc2vec or tfidfvectorizer. To convert the text into numeric data.

Doc2Vec

In [ ]:
import smart_open
import gensim
from gensim import utils
from gensim.models import Doc2Vec

from sklearn.model_selection import train_test_split
X = questions
y = answers
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_list = list()
for i, s in enumerate(X_train):
    tokens = gensim.utils.simple_preprocess(s)
    p_tokens = gensim.models.doc2vec.TaggedDocument(tokens, [i])
    train_list.append(p_tokens)

print(train_list)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_list)

tfidfvectorizer:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = questions
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
X = vectorizer.fit(text)
# summarize
#print(vectorizer.vocabulary_)
print(X.idf_)
# encode document
vector = X.transform([text[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

bigv = X.transform(questions)

list_of_vectors = list()

for t in text:
    vector = X.transform([t])
    array = vector.toarray()
    list_of_vectors.append(array)
    

I have a better grasp of tfidvectorizer, and the frequency weighting is supposed to be better so I will use this method going forward. Next I will test and implement a k-means algorithm. Early on in this assingment I tested an alternative method using gensim and a LDA model, which you can see at the bottom of the document with the rest of my failed experiments, unfortunatly I couldn't get everything working satifactory.

In [ ]:
from sklearn.cluster import KMeans

true_k = 21
model = KMeans(n_clusters=true_k)
model.fit(bigv)

For the clutering algorithm my researched suggested k-means was a solid method, I then experimented with kmeans and minibatchkmeans and their inputs to find the best result.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def find_optimal_clusters(data, max_k):
    iters = range(40, max_k+1, 2)
    
    sse = []
    for k in iters:
        sse.append(KMeans(n_clusters=k, random_state=40).fit(data).inertia_)
        print('Fit {} clusters'.format(k))
        #sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=1024, random_state=40).fit(data).inertia_)
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot K-means')
    
find_optimal_clusters(bigv, 60)

With our ideal k-means cluster value found I can now classify these tweets into 60 categories!!! Hopefully this large size with enable somewhat accurate responses! It turns out k-means was slightly superior over minibatchkmeans. 

Converting each question to a tfdi vector and using my k-means model to assign them a category (cluster).

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=60, random_state=42)
model.fit(bigv)

list_of_clusters = list()

for q in questions:
    vector = X.transform([q])
    cluster = model.predict(vector)
    list_of_clusters.append(cluster)
    
#print(list_of_clusters)
print(len(list_of_clusters))

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords

With my clusters assigned there was the matter of sorting these question-answer-cluster trios into groups for my chatbot to use. I used lists because they seemed like the easiest implementation. Basically I created lists of answers for each cluster id and then combined these into a list of lists. 

In [ ]:
#more sophisticated retireval system
list_of_lists = list()
list_of_ans = list()

i = 0
i_list = list()
while i < 60:
    i_list.append(i)
    i = i + 1

print(i_list)
    

for i in i_list:
    #print(i)    
    
    for q in questions:        
        index = questions.index(q)
        #print(index)
        clusternumber = int(list_of_clusters[index])
        on_cluser = int(i)
        #print(clusternumber)
        if clusternumber == on_cluser:                      
            list_of_ans.append(answers[index])  
    
    #print(list_of_ans)
    #print(i)
    list_of_lists.append(list(list_of_ans))
    #print(list_of_lists)
    list_of_ans.clear()
       
#print(len(list_of_lists))
#print(list_of_lists)

Splitting my data for training/testing and fitting the multinominal naive bayes model - which from my research seemed most appropriate. I also tested guassian (which is why the model name is GNB).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(questions, list_of_clusters, test_size=0.2,random_state=42)

X_traintfv = X.transform(X_train)
X_testtfv = X.transform(X_test)
X_traintfv_array = X_traintfv.toarray()
X_testtfv_array = X_testtfv.toarray()


modelGNB = MultinomialNB()
modelGNB.fit(X_traintfv_array,y_train)
#predictions = modelGNB.predict_proba(X_testtfv_array)

Testing the process my bot with use to understand user input. More details about this below.

In [ ]:
from random import seed
from random import randint
def process_string(user_input):
    nlp = spacy.load("en_core_web_sm")
    my_list = ""
    doc = nlp(user_input)   
    for token in doc:        
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
            my_list += thisdict["lem"] + ' '
       
    return my_list


user_response = 'company you are so bad at customer response'
processed_input = process_string(user_response)
tfdi_input = X.transform([processed_input])
tfdi_input_array = tfdi_input.toarray()

prediction = modelGNB.predict(tfdi_input_array)
print(prediction)

prediction = int(prediction)
print(prediction)
        
possible_answers = list_of_lists[prediction]
print(len(possible_answers))
value = randint(0, len(possible_answers))
        
response = possible_answers[value]
print(response)

This is my second chatbot iteration: this time featuring k-means clustering and multinomial naive bayes for classification.

1) take input
2) preprocess 
3) turn into tfid vector
4) to array
5) predict with multiNB

In [ ]:
import spacy
from random import seed
from random import randint
#from spacy import en_core_web_sm
flag = True


def process_string(user_input):
    nlp = spacy.load("en_core_web_sm")
    my_list = ""
    doc = nlp(user_input)   
    for token in doc:        
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
            my_list += thisdict["lem"] + ' '
       
    return my_list

print('BOT: My name is ChatChat.v2, I will answer your questions about as best as i can. If you want to exit, type Bye')

while flag:
    user_response = input( 'Enter question : ' )
    user_response = user_response.lower()
    print("User: ", user_response)
    answered = 0
    if user_response is not 'bye':
        processed_input = process_string(user_response)
        tfdi_input = X.transform([processed_input])
        tfdi_input_array = tfdi_input.toarray()
        prediction = modelGNB.predict(tfdi_input_array)
        prediction = int(prediction)
        
        possible_answers = list_of_lists[prediction]
        value = randint(0, len(possible_answers))
        
        response = possible_answers[value]
        print(response)
              
    else:
        print('ChatChat: bye!')
        flag = False

Now for deep learning round 20. This is the first seq2seq model I got to work but about my fourth attempt. To start I will edit down the input strings to a certain token length. 

In [ ]:
sentences = list(question_f['processing_inti'].values)
answers_pre =  list(question_f['processing_resp'].values)
max_length = 21
min_length = 2
questions = list()
answers = list()
for sent in sentences:
    length = len(sent.split())
    #print(len(tokens))
    if length > min_length & length < max_length:
        index = sentences.index(sent)
        questions.append(sentences[index])
        answers.append(answers_pre[index])

pre_answer = answers

I first built a joint question-answer vocabulary, then using the built in tokenizer and one hot encoding to prepare the training data for the seq2seq model. (includes pading etc)

In [ ]:
from tensorflow.keras import preprocessing , utils

answers = list()
for i in range( len( pre_answer ) ) :
    answers.append( '<START> ' + pre_answer[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

This is a functional seq2seq model.

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

In [ ]:
del answer_f
del question_f
del full_sample

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=70 )

The inference model to retrieve output:

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


This is a function to prepare user input for the model.

In [ ]:
def str_to_tokens( sentence : str ):
    
    nlp = spacy.load("en_core_web_sm")
    my_list = ""
    doc = nlp(sentence)   
    for token in doc:        
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
            my_list += thisdict["lem"] + ' '
    words = my_list.lower().split()
    tokens_list = list()
    for word in words:
        if tokenizer.word_index[ word ] > 0 & tokenizer.word_index[ word ] < VOCAB_SIZE:
            tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

My 3rd chatbot: featuring a seq2seq model. 

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


With this functional model complete I will try a sequential model with a glove vector embedding. First I will edit down my training data. I didn't end up using this model because I couldn't load as many rows of data to train the model, thus it performed worse. 

In [ ]:
sentences = list(question_f['processing_inti'].values)
answers_pre =  list(question_f['processing_resp'].values)
max_length = 21
min_length = 2
questions = list()
answers = list()
for sent in sentences:
    length = len(sent.split())
    #print(len(tokens))
    if length > min_length & length < max_length:
        index = sentences.index(sent)
        questions.append(sentences[index])
        answers.append(answers_pre[index])

pre_answer = answers

The same preprocessing as before:

In [ ]:
from tensorflow.keras import preprocessing , utils

answers = list()
for i in range( len( pre_answer ) ) :
    answers.append( '<START> ' + pre_answer[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

Get the embedding ready, I decided to go with the 840/300 as a suitible middle ground. 

In [ ]:
EMBEDDING_FILE = '/kaggle/input/glove840b300dtxt/glove.840B.300d.txt'

def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
max_features = len(word_index) - 1
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=20,
                     trainable=True))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback

#model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          #verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=80 )

This is my bleu solution evaluation code

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import spacy



def convert_to_machine(test_input):
    nlp = spacy.load("en_core_web_sm")
    my_list = ""
    doc = nlp(test_input)   
    for token in doc:        
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & (thisdict["pos"] != 'PUNCT') & (thisdict["pos"] != 'SYM'):
            my_list += thisdict["lem"] + ' '
       
    return my_list
#test 1
being_tested = 'User shoot dm look forward'
test1 = convert_to_machine('good how are you')
test2 = convert_to_machine('how can I help you')
test3 = convert_to_machine('please send us a dm we look forward to helping you')

score1 = sentence_bleu(test1, being_tested)
score2 = sentence_bleu(test2, being_tested)
score3 = sentence_bleu(test3, being_tested)

list1 = [score1, score2, score3]
res_max1 = max(list1,key=lambda x:float(x))
print('test 1', res_max1)

#test 2
being_tested = 'User happy help way start want receive detail link hyperlink'
test1 = convert_to_machine('We’re happy to help send us your details ')
test2 = convert_to_machine('Sorry to hear that, can you come into a store so we can look at it')
test3 = convert_to_machine('What ios version are you running?')

score1 = sentence_bleu(test1, being_tested)
score2 = sentence_bleu(test2, being_tested)
score3 = sentence_bleu(test3, being_tested)

list1 = [score1, score2, score3]
res_max2 = max(list1,key=lambda x:float(x))
print('test 2',res_max2)

#test 3
being_tested = 'User forward enjoy flight sabi'
test1 = convert_to_machine('We can absolutely help you, please dm us your details so we can assist you.')
test2 = convert_to_machine('Forward your details to our dms. ')
test3 = convert_to_machine('Yes we can please enjoy your flight.')

score1 = sentence_bleu(test1, being_tested)
score2 = sentence_bleu(test2, being_tested)
score3 = sentence_bleu(test3, being_tested)

list1 = [score1, score2, score3]
res_max3 = max(list1,key=lambda x:float(x))
print('test 3',res_max3)

#test 4
being_tested = 'User welcome anytime end'
test1 = convert_to_machine('You are welcome. We’re happy to help anytime.')
test2 = convert_to_machine('Thank you let us know if you have any further queries.')
test3 = convert_to_machine('Please reach out to us if you have any further concerns.')

score1 = sentence_bleu(test1, being_tested)
score2 = sentence_bleu(test2, being_tested)
score3 = sentence_bleu(test3, being_tested)

list1 = [score1, score2, score3]
res_max4 = max(list1,key=lambda x:float(x))
print('test 4',res_max4)

#test 5
being_tested = 'User need assist complete service address need help hyperlink'
test1 = convert_to_machine('User loreen kindly dm book number attach guest passport copy fix there! - anwar')
test2 = convert_to_machine('Do you need help?')
test3 = convert_to_machine('We’re sorry to hear that, can you please dm us about your service today?')

score1 = sentence_bleu(test1, being_tested)
score2 = sentence_bleu(test2, being_tested)
score3 = sentence_bleu(test3, being_tested)

list1 = [score1, score2, score3]

res_max5 = max(list1,key=lambda x:float(x))
print('test 5',res_max5)

BLEU_score = res_max1 + res_max2 + res_max3 + res_max4 + res_max5
BLEU_score = BLEU_score / 5
print('Blue score', BLEU_score)

This is my solution evaluation using an LSA score

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")


def convert_to_machine(test_input):
    nlp = spacy.load("en_core_web_sm")
    my_list = ""
    doc = nlp(test_input)   
    for token in doc:        
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & (thisdict["pos"] != 'PUNCT') & (thisdict["pos"] != 'SYM'):
            my_list += thisdict["lem"] + ' '
       
    return my_list
#test 1
being_tested = 'User shoot dm look forward end'
test1 = convert_to_machine('good how are you')
test2 = convert_to_machine('how can I help you')
test3 = convert_to_machine('please send us a dm we look forward to helping you')

being_tested = nlp(being_tested)
test1 = nlp(test1)
test2 = nlp(test2)
test3 = nlp(test3)

simil1 = being_tested.similarity(test1)
simil2 = being_tested.similarity(test2)
simil3 = being_tested.similarity(test3)

list1 = [simil1, simil2, simil3]
res_max1 = max(list1,key=lambda x:float(x))
print(res_max1)

#test 2
being_tested = 'User happy help way start want receive detail link hyperlink end'
test1 = convert_to_machine('We’re happy to help send us your details ')
test2 = convert_to_machine('Sorry to hear that, can you come into a store so we can look at it')
test3 = convert_to_machine('What ios version are you running?')

being_tested = nlp(being_tested)
test1 = nlp(test1)
test2 = nlp(test2)
test3 = nlp(test3)

simil1 = being_tested.similarity(test1)
simil2 = being_tested.similarity(test2)
simil3 = being_tested.similarity(test3)

list1 = [simil1, simil2, simil3]
res_max2 = max(list1,key=lambda x:float(x))
print(res_max2)

#test 3
being_tested = 'User forward enjoy flight sabi end'
test1 = convert_to_machine('We can absolutely help you, please dm us your details so we can assist you.')
test2 = convert_to_machine('Forward your details to our dms. ')
test3 = convert_to_machine('Yes we can please enjoy your flight.')

being_tested = nlp(being_tested)
test1 = nlp(test1)
test2 = nlp(test2)
test3 = nlp(test3)

simil1 = being_tested.similarity(test1)
simil2 = being_tested.similarity(test2)
simil3 = being_tested.similarity(test3)

list1 = [simil1, simil2, simil3]
res_max3 = max(list1,key=lambda x:float(x))
print(res_max3)

#test 4
being_tested = 'User welcome anytime end'
test1 = convert_to_machine('You are welcome. We’re happy to help anytime.')
test2 = convert_to_machine('Thank you let us know if you have any further queries.')
test3 = convert_to_machine('Please reach out to us if you have any further concerns.')

being_tested = nlp(being_tested)
test1 = nlp(test1)
test2 = nlp(test2)
test3 = nlp(test3)

simil1 = being_tested.similarity(test1)
simil2 = being_tested.similarity(test2)
simil3 = being_tested.similarity(test3)

list1 = [simil1, simil2, simil3]
res_max4 = max(list1,key=lambda x:float(x))
print(res_max4)

#test 5
being_tested = 'User need assist complete service address need help hyperlink end'
test1 = convert_to_machine('Sorry to hear that, how can we help you?')
test2 = convert_to_machine('Do you need help?')
test3 = convert_to_machine('We’re sorry to hear that, can you please dm us about your service today?')

being_tested = nlp(being_tested)
test1 = nlp(test1)
test2 = nlp(test2)
test3 = nlp(test3)

simil1 = being_tested.similarity(test1)
simil2 = being_tested.similarity(test2)
simil3 = being_tested.similarity(test3)

list1 = [simil1, simil2, simil3]
res_max5 = max(list1,key=lambda x:float(x))
print(res_max5)

LSA_score = res_max1 + res_max2 + res_max3 + res_max4 + res_max5
LSA_score = LSA_score / 5
print('lsa score: ', LSA_score)

Below are some of my failed experiments, I wanted to separate them from my main project to not risk cluttering the kaggle document. I'll try to comment the separate projects but it's a bit of a mess. I don't expect most of this to run 100%.

This was some of my experimenting with word vectors and gensim as an alternative to the k-means algorithm. 

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


pre_data = question_f['processing_inti'].values

data = list(sent_to_words(pre_data))

#print(data)
id2word = corpora.Dictionary(data)

texts = data

corpus = [id2word.doc2bow(text) for text in texts]

#print(corpus)
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]



lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#print(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

This was the RNN model I orginally intended to use, if I could've gotten it to work effectively.

In [ ]:
		
import tensorflow as tf
import tensorflow.compat.v1 as v1
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = v1.placeholder(tf.int32, [None, None], name='input')
    targets = v1.placeholder(tf.int32, [None, None], name='targets')
    lr = v1.placeholder(tf.float32, name='learning_rate')
    keep_prob = v1.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob
	import tensorflow as tf
import tensorflow.compat.v1 as v1
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = v1.placeholder(tf.int32, [None, None], name='input')
    targets = v1.placeholder(tf.int32, [None, None], name='targets')
    lr = v1.placeholder(tf.float32, name='learning_rate')
    keep_prob = v1.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob
	def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length, attn_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length, state_is_tuple = True)
    enc_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)

    return enc_state
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length, attn_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length, state_is_tuple = True)
    enc_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)

    return enc_state
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    '''Decode the prediction data'''
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(
        output_fn, encoder_state, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size)
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)
    return infer_logits
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, attn_length):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length, state_is_tuple = True)
        dec_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(
            encoder_state[0], dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state[0], dec_cell, dec_embeddings, vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], sequence_length, vocab_size,
                                            decoding_scope, output_fn, keep_prob)

    return train_logits, infer_logits
from tensorflow.python.compiler import tensorrt as trt

def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, vocab_to_int, attn_length):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = trt.contrib.layers.embed_sequence(input_data, source_vocab_size+1, enc_embedding_size)
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length, attn_length)

    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size+1, dec_embedding_size], -1.0, 1.0))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    train_logits, infer_logits = decoding_layer(dec_embed_input, dec_embeddings, enc_state, target_vocab_size+1, 
                                                sequence_length, rnn_size, num_layers, vocab_to_int, keep_prob, 
                                                attn_length)
    
    return train_logits, infer_logits
# Set the parameters


epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
attn_length = 10
learning_rate = 0.0005
keep_probability = 0.8
max_line_length = 30

train_graph = tf.Graph()
with train_graph.as_default():
    
    # Load the model inputs
    input_data, targets, lr, keep_prob = model_inputs()
    # Sequence length will be the max line length for each batch
    sequence_length = max_line_length
    input_shape = tf.shape(input_data)
    
    # Create the logits from the model
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), 
        len(target_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
        target_vocab_to_int, attn_length)
    
    # Create a tensor to be used for making predictions.
    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

Some testing of word2vec.

In [ ]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary



Testing the glove embedding and some different implementations.

In [ ]:
max_features = 10000
maxlen = 20

X_train, X_test, y_train, y_test = train_test_split(questions, answers, test_size=0.2,random_state=42)

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
EMBEDDING_FILE = '/kaggle/input/glove840b300dtxt/glove.840B.300d.txt'

def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
#EMBEDDING_FILE = '/kaggle/input/glove840b300dtxt/glove.840B.300d.txt'

embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt', errors = 'ignore', encoding='utf8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
xtrain_glove = [sent2vec(x) for x in tqdm(X_train)]
xvalid_glove = [sent2vec(x) for x in tqdm(X_test)]

Testing of tfidvectorizer.

In [ ]:
from gensim import utils
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
def get_tfdi(text1):
    tfidf = vectorizer.fit_transform([text1])
    return text1

In [ ]:
Tfidf_scores = list()
for s in questions:
    score = get_tfdi(s)
    #print(s)
    #print(score)
    Tfidf_scores.append(score)

In [ ]:
print(Tfidf_scores)

Testing of word vectors and learning what they are.

In [ ]:
import gensim.models

sentences = questions_tokened
model = gensim.models.Word2Vec(sentences=sentences, vector_size=300)

In [ ]:
vec_customer = model.wv['customer']
#print(vec_customer)

In [ ]:
model.wv.most_similar(positive=['customer'])

In [ ]:
print('customer', 'bad', model.similarity('customer', 'bad'))

In [ ]:
w2v_model = word2vec_model()
w2v_model.save('word2vec_model')

In [ ]:
print('customer', 'bad', w2v_model.similarity('customer', 'bad'))

Another method for seq2seq modelling.

In [ ]:
sentences = list(question_f['processing_inti'].values)
answers_pre =  list(question_f['processing_resp'].values)
max_length = 21
min_length = 2
questions = list()
answers = list()
for sent in sentences:
    length = len(sent.split())
    #print(len(tokens))
    if length > min_length & length < max_length:
        index = sentences.index(sent)
        questions.append(sentences[index])
        answers.append(answers_pre[index])
        
print(len(questions))
print(len(answers))

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Alternative method to preping the data for the seq2seq model featuring a different method to build a vocab.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X = questions
y = answers
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

max_features = 10000
maxlen = 20
embed_size = 300

threshold = 0.35

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts((X_train) + (X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
x_test = np.asarray(x_test).astype('float32')
X_train = np.asarray(X_train).astype('float32')
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [ ]:
vocab_a = {}
def build_vocaba(potential):
    for line in potential:
        for word in line.split():
            if word not in vocab_a:
                vocab_a[word] = 1
            else:
                vocab_a[word] += 1

build_vocaba(answers)

threshold = 2
count = 0
for k,v in vocab_a.items():
    if v >= threshold:
        count += 1
        
print("Size of total vocab:", len(vocab_a))
print("Size of vocab we will use:", count)

decoder_vocab = len(vocab_a)

In [ ]:
vocab_q = {}
def build_vocabq(potential):
    for line in potential:
        for word in line.split():
            if word not in vocab_q:
                vocab_q[word] = 1
            else:
                vocab_q[word] += 1

build_vocabq(questions)

threshold = 2
count = 0
for k,v in vocab_q.items():
    if v >= threshold:
        count += 1
        
print("Size of total vocab:", len(vocab_q))
print("Size of vocab we will use:", count)

encoder_vocab = len(vocab_q)

In [ ]:
encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(
    encoder_input
)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(
    encoder_embedded
)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(
    decoder_input
)

# Pass the 2 states to a new LSTM layer, as initial state
decoder_output = layers.LSTM(64, name="decoder")(
    decoder_embedded, initial_state=encoder_state
)
output = layers.Dense(10)(decoder_output)

model = keras.Model([encoder_input, decoder_input], output)
model.summary()

In [ ]:
batch_size = 64

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

model.fit(
    X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=10
)

Alternative method for preparing the data, and a model at the end.

I will now prepare the tokenized data ready for the seq2seq
First I prepared encoder_input_data:
1. Determining the maximum length of the question sentences that's max_input_length.
2. Padding the tokenized_que_lines to the max_input_length.
3. Determining the vocabulary size ( num_que_tokens ) 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils

#question_lines = question_f['processing_inti']

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions ) 
tokenized_que_lines = tokenizer.texts_to_sequences( questions )

length_list = list()
for token_seq in tokenized_que_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Question max length is {}'.format( max_input_length ))

padded_ques_lines = preprocessing.sequence.pad_sequences( tokenized_que_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_ques_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

que_word_dict = tokenizer.word_index
num_que_tokens = len( que_word_dict )+1
print( 'Number of Question tokens = {}'.format( num_que_tokens))

Next decoder_input_data:
1. Append '<'START'>' and '<'END'>' to each of the answer sentences.
2. Same steps as before

In [ ]:
#pre_answer_lines = question_f['processing_resp']
answer_lines = list()

for line in answers:
    answer_lines.append( '<START' + line + '<END>' )
    
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( answer_lines ) 
tokenized_ans_lines = tokenizer.texts_to_sequences( answer_lines ) 

length_list = list()
for token_seq in tokenized_ans_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Answer max length is {}'.format( max_output_length ))

padded_ans_lines = preprocessing.sequence.pad_sequences( tokenized_ans_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_ans_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

ans_word_dict = tokenizer.word_index
num_ans_tokens = len( ans_word_dict )+1
print( 'Number of Answer tokens = {}'.format( num_ans_tokens))

create a vocab and get size

In [ ]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( answers + questions )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

Prepare decoder_target_data:
1. remove '<'START'>' tokens
2. one-hot encoding 

In [ ]:
decoder_target_data = list()
for token_seq in tokenized_ans_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_ans_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_ans_lines = utils.to_categorical( padded_ans_lines , VOCAB_SIZE )
decoder_target_data = np.array( onehot_ans_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(
    encoder_input
)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(
    encoder_embedded
)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(
    decoder_input
)

# Pass the 2 states to a new LSTM layer, as initial state
decoder_output = layers.LSTM(64, name="decoder")(
    decoder_embedded, initial_state=encoder_state
)
output = layers.Dense(10)(decoder_output)

model = keras.Model([encoder_input, decoder_input], output)
model.summary()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

data = questions
layer = preprocessing.TextVectorization()
layer.adapt(data)
vectorized_textq = layer(data)
print(vectorized_textq)

data = answers
layer = preprocessing.TextVectorization()
layer.adapt(data)
vectorized_texta = layer(data)
print(vectorized_texta)

In [ ]:
from sklearn.model_selection import train_test_split

X = vectorized_textq
y = vectorized_texta

X_train, X_test, y_train, y_test = train_test_split(vectorized_textq, vectorized_texta, test_size=0.2, random_state=42)

model = build_model(allow_cudnn_kernel=True)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=10
)

More model sequencing.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

In [ ]:
def seq2seq_model_builder(HIDDEN_DIM=300):
    
    encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    encoder_embedding = embed_layer(encoder_inputs)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
    decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    decoder_embedding = embed_layer(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
    # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    
    return model

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_que_tokens))
encoder = LSTM(VOCAB_SIZE, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_ans_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(num_que_tokens, VOCAB_SIZE)(encoder_inputs)
x, state_h, state_c = LSTM(VOCAB_SIZE,
                           return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(num_ans_tokens, VOCAB_SIZE)(decoder_inputs)
x = LSTM(VOCAB_SIZE, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:

encoder_inputs = tf.keras.layers.Input(shape=( max_input_length , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True  )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( max_output_length ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=50, epochs=150 )
#model.save( 'model.h5' ) 